<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Implementation of the NRPyElliptic solver in ***curvilinear*** coordinates, using a reference metric formalism


## Author: Thiago Assumpção

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation).

### NRPy+ Source Code for this module: [NRPyElliptic_codegen/NRPyElliptic_RHSs.py](../edit/NRPyElliptic/NRPyElliptic_RHSs.py) and [NRPyElliptic_codegen/NRPyElliptic_SourceTerm.py](../edit/NRPyElliptic/NRPyElliptic_SourceTerm.py)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

0. [Introduction](#intro): The ``NRPyElliptic`` code
    1. [Intro. a](#initial_data): Basic equations
    1. [Intro. b](#hyper_relax): Hyperbolized Hamiltonian constraint equation
1. [Step 1](#params): Importing necessary modules and defining physical parameters
1. [Step 2](#coord): Setting up coordinate system and reference metric
1. [Step 3](#add_times_auu): Computing contraction of conformal extrinsic curvature $\tilde{A}_{ij}\tilde{A}^{ij}$ 
    1. [Step 3.a](#vu): Computing vector field $\vec{V}$
    1. [Step 3.b](#add): Computing extrinsic curvature $\tilde{A}_{ij}$
    1. [Step 3.c](#add_auu): Computing $\tilde{A}_{ij} \tilde{A}^{ij}$
1. [Step 4](#psi_singular): Computing $\psi_{\rm singular}$
1. [Step 5](#rhss): Computing the RHSs
1. [Step 6](#code_validation): Code validation against `NRPyElliptic.NRPyElliptic_RHSs` and `NRPyElliptic.NRPyElliptic_SourceTerm`  NRPy+ modules
    1. [Step 6.a](#code_validation_sourceterm): Code validation against `NRPyElliptic.NRPyElliptic_SourceTerm` NRPy+ module 
    1. [Step 6.b](#code_validation_rhss): Code validation against `NRPyElliptic.NRPyElliptic_RHSs` NRPy+ module   
1. [Step 7](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='intro'></a>

# Introduction: The ``NRPyElliptic`` code \[Back to [top](#toc)\]
$$\label{intro}$$

``NRPyElliptic`` [[Assumpcao et. al., arXiv:gr-qc/2111.02424](https://arxiv.org/abs/2111.02424)] is a new, extensible elliptic solver that sets up initial data (ID) for numerical relativity (NR) using the same numerical methods employed for solving hyperbolic evolution equations. Specifically, ``NRPyElliptic`` implements the hyperbolic relaxation method of [[Rüter et. al., arXiv:gr-qc/1708.07358](https://arxiv.org/abs/1708.07358)] to solve complex nonlinear elliptic PDEs for NR ID. The hyperbolic PDEs are evolved forward in (pseudo)time, resulting in an exponential relaxation of the arbitrary initial guess to a steady state that coincides with the solution of the elliptic system. ``NRPyElliptic`` solves these equations on highly efficient numerical grids exploiting underlying symmetries in the physical scenario. To this end, ``NRPyElliptic`` is built within the [SymPy](https://peerj.com/articles/cs-103/)-based [``NRPy+``](https://nrpyplus.net/) framework, which facilitates the solution of hyperbolic PDEs on Cartesian-like, spherical-like, cylindrical-like, or bispherical-like numerical grids. For the purposes of setting up BBH puncture ID, ``NRPyElliptic`` makes use of the latter.

<a id='initial_data'></a>

# Puncture initial data: basic equations \[Back to [top](#toc)\]
$$\label{initial_data}$$

For completeness, I include a brief description of binary pucture initial data below. Feel free to skip this section if you are only interested in the implementation of the equations.

$\newcommand{\ttilde}[1]{\mathbf{\tilde{\text{$#1$}}}}$
In the limit of vacuum (e.g., BBH) spacetimes, the Hamiltonian
and momentum constraint equations can be written as (see, e.g., [Cook, arXiv:gr-qc/0007085](https://arxiv.org/abs/gr-qc/0007085) and references therein)
\begin{align}
\label{eqn:hamiltonian_constraint}
\mathcal{H} &\equiv R + K^2 - K_{ij} K^{ij} = 0 \,, \\
\label{eqn:momentum_constraint}
\mathcal{M}^{i} &\equiv \nabla_{j}(K^{ij} - \gamma^{ij}K) = 0 \,,
\end{align}
where $\gamma_{ij}$ is the 3-metric, $R$ is the 3-Ricci scalar, $\nabla_{i}$ is the covariant derivative compatible with the 3-metric, $K_{ij}$ is the extrinsic curvature, and $K\equiv\gamma^{ij}K_{ij}$
is the mean curvature. Setting up ID for vacuum spacetimes in numerical
relativity generally involves solving these constraints, which exist as second-order nonlinear
elliptic PDEs.

For the purposes of this tutorial, we will focus on the puncture ID
formalism, in which a set of simplifying assumptions is applied to these
constraints, known as the conformal transverse-traceless (CTT) decomposition

<a id='punct_ID_and_CTT'></a>

## Puncture initial data and the conformal transverse-traceless (CTT) formalism \[Back to [top](#toc)\]
$$\label{punct_ID_and_CTT}$$

It is useful to define a
conformally related $3$-metric $\ttilde{\gamma}_{ij}$ via
\begin{equation}
\gamma_{ij} = \psi^4 \ttilde{\gamma}_{ij} \,,
\end{equation}
where the scalar function $\psi$ is known as the conformal factor. We adorn geometric quantities associated with $\ttilde{\gamma}_{ij}$ with a tilde diacritic.

Under the CTT formalism, the Hamiltonian and momentum constraint equations become  (see, e.g., [Numerical Relativity: Solving Einstein's Equations on the Computer](https://www.cambridge.org/us/academic/subjects/physics/astrophysics/numerical-relativity-solving-einsteins-equations-computer?format=HB) for a pedagogical introduction)
\begin{align}
\ttilde{\nabla}^2 \psi - \frac{1}{8}\psi \ttilde{R} - \frac{1}{12}\psi^5 K^2 + \frac{1}{8}\psi^{-7} \ttilde{A}_{ij} \ttilde{A}^{ij} &= 0 \,, \\
\ttilde{\Delta}_{\mathbb{L}} V^i - \frac{2}{3} \psi^6 \ttilde{\nabla}^i K + \ttilde{\nabla}_j \ttilde{M}^{ij} &= 0 \,,\label{eq:MU_cttd}
\end{align}

where $\ttilde{R}$ is the conformal Ricci scalar and the operator $\ttilde{\Delta}_{\mathbb{L}}$ is defined as

\begin{equation}
\ttilde{\Delta}_{\mathbb{L}} V^i \equiv \ttilde{\nabla}_j (\ttilde{\mathbb{L}} V)^ {ij} =  \ttilde{\nabla}^2 V^i + \frac{1}{3} \ttilde{\nabla}^i(\ttilde{\nabla}_j V^j) + \ttilde{R}^{i}_{\ j} V^j \,.
\end{equation}

The quantity $\ttilde{A}_{ij} \equiv \psi^{2}{A}_{ij}$ is the conformal traceless extrinsic curvature, where 
$$
K_{ij} = A_{ij} + \frac{1}{3} \gamma_{ij} K \,.
$$



The degrees of freedom in this formulation include choice of $\ttilde{M}^{ij}$, $K$, and $\ttilde\gamma_{ij}$. Here we consider puncture ID, which assume maximal slicing ($K = 0$),
asymptotic flatness ($\psi|_{r\to\infty}=1$), and conformal flatness
\begin{equation}
\ttilde\gamma_{ij} = \hat\gamma_{ij}\,,
  \label{eq:BY_confflatness}
\end{equation}

where $\hat\gamma_{ij}$ is the flat spatial metric. In addition the assumption
$\ttilde{M}^{ij} = 0$ is made, yielding Hamiltonian and momentum constraint equations of the form [[Ansorg et. al., arXiv:gr-qc/0404056](https://arxiv.org/abs/gr-qc/0404056)]
\begin{align}
\label{eqn:hamiltonian_constraint_v2}
\hat{\nabla}^2 \psi + \frac{1}{8}\psi^{-7} \ttilde{A}_{ij} \ttilde{A}^{ij} &= 0 \,, \\
\label{eqn:momentum_constraint_v2}
\hat{\nabla}^2 V^i + \frac{1}{3} \hat{\nabla}^i (\hat{\nabla}_j V^j) + \hat{R}^{i}_{\ j} V^j &= 0\,,
\end{align}

where $\hat\nabla_{i}$ is the covariant derivative compatible with
$\hat\gamma_{ij}$. Bowen and York [[Phys. Rev. D 21, 2047 (1980)](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.21.2047)] showed hat the
momentum constraint is solved
for a set of $N_p$ punctures with a
closed-form expression for the extrinsic curvature. This expression
can be written in terms of $\vec{V}$ as follows

\begin{equation}
    \vec{V} = \sum_{n=1}^{N_p}
\left(
- \frac{7}{4 |\vec{x}_n|} \vec{P}_n
- \frac{\vec{x}_n {\cdot} \vec{P}_n}{4 |\vec{x}_n|^3} \vec{x}_n
+ \frac{1}{|\vec{x}_n|^3} \vec{x}_n {\times} \vec{S}_n
\right),
\end{equation}

where $\vec{x}_n = (x_n - x, y_n - y, z_n - z)$, $\vec{P}_n$, and
$\vec{S}_n$ are the displacement relative to the origin (i.e., $(x,y,z)=(0,0,0)$), linear momentum, and
spin angular momentum of puncture $n$, respectively.

The Hamiltonian constraint equation must be solved numerically, but $\psi$ becoming singular at
the location of each puncture could spoil the numerical solution. We split the
conformal factor into a singular and a non-singular piece [[Phys. Rev. Lett. 78, 3606 (1997)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.78.3606)],

\begin{equation}
\psi = \psi_{\rm singular} + u \equiv 1 + \sum_{n=1}^{N_p} \frac{m_{n}}{2|\vec{x}_{n} |} + u \,,
\end{equation}

where $m_n$ is the bare mass of the $n^{\rm{th}}$ puncture. The
Hamiltonian constraint equation, which can then be solved for the
non-singular part $u$, reads

\begin{equation}
\hat{\nabla}^2 u + \frac{1}{8} \ttilde{A}_{ij} \ttilde{A}^{ij} (\psi_{\rm singular} +  u)^{-7} = 0 \,,
\end{equation}

since the Laplacian of the singular piece vanishes.

<a id='hyper_relax'></a>

# Hyperbolic relaxation method: Hyperbolized Hamiltonian constraint equation \[Back to [top](#toc)\]
$$\label{hyper_relax}$$

A complete description of the method can be found at [[Assumpcao et. al., arXiv:gr-qc/2111.02424](https://arxiv.org/abs/2111.02424)] or [[Rüter et. al., arXiv:gr-qc/1708.07358](https://arxiv.org/abs/1708.07358)]. The hyperbolized Hamiltonian constraint equation is

\begin{equation}
  \boxed{
    \begin{aligned}
      \partial_{t}u &= v - \eta u\\
      \partial_{t}v &= c^{2}\biggl[\ttilde\nabla^{2}u + \frac{1}{8}\ttilde{A}_{ij}\ttilde{A}^{ij}\bigl(\psi_{\rm singular}+u\bigr)^{-7}\biggr]
    \end{aligned}
  }\ ,
\end{equation}

where $\eta$ is the damping parameter, $c$ is the wavespeed, and $v$ is an auxiliary variable so that the equation can be expressed as a first-order system. In the steady-state regime, in which $\partial_{t}u = \partial_{t}v = 0$, we recover the solution to the original elliptic equation (the RHS of the second equation).

<a id='params'></a>

# Step 1: Importing necessary modules and defining physical parameters \[Back to [top](#toc)\]
$$\label{params}$$

In [1]:
# Import needed NRPy+ core modules:
import NRPy_param_funcs as par                # NRPy+: Parameter interface
import grid as gri                            # NRPy+: Functionality for handling numerical grids
import indexedexp as ixp                      # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm                # NRPy+: Reference metric support
import sympy as sp                            # SymPy: The Python computer algebra package upon which NRPy+ depends

In [2]:
# Set dimension
DIM = 3

In [3]:
# Set module name
thismodule = __name__
# Step 1a:  Define the C parameters. The variables are 
#           proper SymPy variables, so they can be
#           used in below expressions. In the C code, they act
#           just like usual parameters, whose value are
#           specified in the parameter file.
# Step 1: Define the C parameters for punctures.
# Step 1a: Define bare masses
bare_mass_0 = par.Cparameters("REAL",thismodule,"bare_mass_0", 1.0)
bare_mass_1 = par.Cparameters("REAL",thismodule,"bare_mass_1", 1.0)
# Step 1b.i: Define puncture 0 positions
puncture_0_x = par.Cparameters("REAL",thismodule,"puncture_0_x", 0.0)
puncture_0_y = par.Cparameters("REAL",thismodule,"puncture_0_y", 0.0)
puncture_0_z = par.Cparameters("REAL",thismodule,"puncture_0_z", 0.0)
puncture_0 = [puncture_0_x, puncture_0_y, puncture_0_z] # define list with punctures
# Step 1b.ii: Define puncture 1 positions
puncture_1_x = par.Cparameters("REAL",thismodule,"puncture_1_x", 0.0)
puncture_1_y = par.Cparameters("REAL",thismodule,"puncture_1_y", 0.0)
puncture_1_z = par.Cparameters("REAL",thismodule,"puncture_1_z", 0.0)
puncture_1 = [puncture_1_x, puncture_1_y, puncture_1_z] # define list with punctures
# Step 1c.i: Define linear momentum 0
P0_x = par.Cparameters("REAL",thismodule,"P0_x", 0.0)
P0_y = par.Cparameters("REAL",thismodule,"P0_y", 0.0)
P0_z = par.Cparameters("REAL",thismodule,"P0_z", 0.0)
P0 = [P0_x, P0_y, P0_z] # define list with linear momenta
# Step 1c.ii: Define linear momentum 1
P1_x = par.Cparameters("REAL",thismodule,"P1_x", 0.0)
P1_y = par.Cparameters("REAL",thismodule,"P1_y", 0.0)
P1_z = par.Cparameters("REAL",thismodule,"P1_z", 0.0)
P1 = [P1_x, P1_y, P1_z] # define list with linear momenta
# Step 1c.iii: Define linear momenta vectors
PU = ixp.zerorank2()
for i in range(DIM):
    PU[0][i] = P0[i]
    PU[1][i] = P1[i]
# Step 1d.i: Define angular momentum 0
S0_x = par.Cparameters("REAL",thismodule,"S0_x", 0.0)
S0_y = par.Cparameters("REAL",thismodule,"S0_y", 0.0)
S0_z = par.Cparameters("REAL",thismodule,"S0_z", 0.2)
S0 = [S0_x, S0_y, S0_z] # define list with angular momenta
# Step 1d.ii: Define angular momentum 1
S1_x = par.Cparameters("REAL",thismodule,"S1_x", 0.0)
S1_y = par.Cparameters("REAL",thismodule,"S1_y", 0.0)
S1_z = par.Cparameters("REAL",thismodule,"S1_z", 0.0)
S1 = [S1_x, S1_y, S1_z] # define list with angular momenta
# Step 1d.iii: Define angular momenta vectors
SU = ixp.zerorank2()
for i in range(DIM):
    SU[0][i] = S0[i]
    SU[1][i] = S1[i]

<a id='coord'></a>

# Step 2: Setting up coordinate system and reference metric \[Back to [top](#toc)\]
$$\label{coord}$$

In [4]:
# Step 2.a: Get the spatial dimension, defined in the
#         NRPy+ "grid" module. With reference metrics,
#         this must be set to 3 or fewer. As default, it is set to 3.
DIM = par.parval_from_str("DIM")
# Step 2.b: Set the coordinate system for the numerical grid
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
CoordSystem     = "SinhSymTP"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
# Step 2.c Compute reference metric quantities
rfm.reference_metric()
# Step 2.d: Define grid
xx = gri.xx
# Step 2.e: Define Cartesian grid
xxCart = rfm.Cart

Here we set the relative positions of each puncture ``xU[n]`` = $\vec{x}_n = (x - x_n, y - y_n, z - z_n)$ in Cartesian coordinates:

In [5]:
# Step 2.f: Set relative positions of each puncture
xU = ixp.zerorank2()
for i in range(DIM):
    xU[0][i] = xxCart[i] - puncture_0[i]
    xU[1][i] = xxCart[i] - puncture_1[i]

<a id='add_times_auu'></a>

# Step 3: Computing contraction of conformal extrinsic curvature $\tilde{A}_{ij}\tilde{A}^{ij}$ \[Back to [top](#toc)\]
$$\label{add_times_auu}$$

Since $\ttilde{A}_{ij}\ttilde{A}^{ij}$ is a scalar quantity, we compute it in Cartesian coordinates.

<a id='vu'></a>

## Step 3.a: Computing vector field $\vec{V}$ \[Back to [top](#toc)\]
$$\label{vu}$$

We first define two functions for computing the dot product and the cross product between two vectors:

In [6]:
# Step 3.a: Define dot and cross product of vectors
def dot(vec1,vec2):
    vec1_dot_vec2 = sp.sympify(0)
    for i in range(3):
        vec1_dot_vec2 += vec1[i]*vec2[i]
    return vec1_dot_vec2
def cross(vec1,vec2):
    vec1_cross_vec2 = ixp.zerorank1()
    LeviCivitaSymbol = ixp.LeviCivitaSymbol_dim3_rank3()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                vec1_cross_vec2[i] += LeviCivitaSymbol[i][j][k]*vec1[j]*vec2[k]
    return vec1_cross_vec2

Defining function for a single puncuture:
\begin{equation}
    \vec{V}_n =
- \frac{7}{4 |\vec{x}_n|} \vec{P}_n
- \frac{\vec{x}_n {\cdot} \vec{P}_n}{4 |\vec{x}_n|^3} \vec{x}_n
+ \frac{1}{|\vec{x}_n|^3} \vec{x}_n {\times} \vec{S}_n \,.
\end{equation}

In [7]:
def VU_cart_single_puncture(xU, PU, SU):
    """
    xnU = position of n-th puncture
    PnU = linear momentum of n-th puncture
    SnU = spin of n-th puncture
    """
    DIM = par.parval_from_str("DIM")
    r = sp.sympify(0)
    for i in range(DIM):
        r += sp.Pow(xU[i], 2)
    r = sp.sqrt(r)

    VU_cart = ixp.zerorank1()
    for i in range(DIM):
        VU_cart[i] += ( sp.Rational(-7, 4) * PU[i] / r
                      + sp.Rational(-1, 4) * dot(xU, PU) * xU[i] / sp.Pow(r, 3)
                      + cross(xU, SU)[i] / sp.Pow(r, 3) )
    return VU_cart

Now we compute the vector field $\vec{V}$ for a superposition of two punctures:
\begin{equation}
    \vec{V} = \sum_{n=1}^{2} \vec{V}_n = \sum_{n=1}^{2}
\left(
- \frac{7}{4 |\vec{x}_n|} \vec{P}_n
- \frac{\vec{x}_n {\cdot} \vec{P}_n}{4 |\vec{x}_n|^3} \vec{x}_n
+ \frac{1}{|\vec{x}_n|^3} \vec{x}_n {\times} \vec{S}_n
\right).
\end{equation}

In [8]:
def VU_cart_two_punctures(x0U, P0U, S0U, x1U, P1U, S1U):
    """
    xnU = position of n-th puncture
    PnU = linear momentum of n-th puncture
    SnU = spin of n-th puncture
    """
    V0U_cart = VU_cart_single_puncture(x0U, P0U, S0U)
    V1U_cart = VU_cart_single_puncture(x1U, P1U, S1U)

    VU_cart = ixp.zerorank1()
    for i in range(DIM):   
        VU_cart[i] = V0U_cart[i] + V1U_cart[i]     
    return VU_cart

<a id='add'></a>

## Step 3.b: Computing extrinsic curvature $\tilde{A}_{ij}$ \[Back to [top](#toc)\]
$$\label{add}$$

Since we chose $K=0$, it follows that $K_{ij} = A_{ij} = \psi^{-2} \tilde{A}_{ij}$. By virtue of Eq. (23) in [Ansorg et. al., arXiv:gr-qc/0404056](https://arxiv.org/abs/gr-qc/0404056), the conformal extrinsic curvature, in Cartesian coordinates, is

$$
    \tilde{A}_{ij} = V_{i,j} + V_{j,i} - \frac{2}{3} \delta_{ij} \text{div} (\vec{V}) \,.
$$

Let us now code up this quantity.

In [9]:
def ADD_conf_cartesian(VU_cart):
    """
    Input: vector VU in Cartesian coordinates

    Output: Components of the conformally-related extrinsic 
            curvature in Cartesian coordinates with two lower indices
    """
    DIM = par.parval_from_str("DIM")

    # Step 1: Define vector field V_i
    #           In Cartesian coordinates, V_i = V^i
    VD_cart = ixp.zerorank1()
    for i in range(DIM):
        VD_cart[i] = VU_cart[i]

    # Step 2: Compute the derivatives: sp.diff(VD_cart[i], xxCart[j])
    VD_cart_dD = ixp.zerorank2()
    for i in range(DIM):
        for j in range(DIM):
            VD_cart_dD[i][j] = sp.diff(VD_cart[i], xxCart[j])

    # Step 3: Compute the divergence
    V_cart_div = sp.sympify(0)
    for i in range(DIM):
        V_cart_div += VD_cart_dD[i][i]

    # Step 4: Define Kronecker delta symbol as a function
    def kronecker_delta(i,j):
        if (i==j):
            return sp.sympify(1)
        else:
            return sp.sympify(0)

    # Step 5: Compute the components of the conformally-related extrinsic curvature
    #           in Cartesian coordinates with two lower indices, and return result
    ADD_conf_cart = ixp.zerorank2()
    for i in range(DIM):
        for j in range(DIM):
            ADD_conf_cart[i][j] = (VD_cart_dD[i][j] + VD_cart_dD[j][i]
                                   - sp.Rational(2,3)*kronecker_delta(i,j)*V_cart_div)
    return ADD_conf_cart

<a id='add_auu'></a>

## Step 3.c: Computing $\tilde{A}_{ij} \tilde{A}^{ij}$ \[Back to [top](#toc)\]
$$\label{add_auu}$$

In a flat three-dimensional space representated by Cartesian coordinates, 

$$
\ttilde{A}^{ij} \equiv \ttilde{A}_{ij} \,.
$$


In [10]:
def ADD_times_AUU_conf_cartesian(ADD_conf_cart):
    """
    Input: Components of the conformally-related extrinsic 
            curvature in Cartesian coordinates with two lower indices

    Output: The contraction ADD_conf_cart[i][j]*ADD_conf_cart[i][j]
    """
    ADD_times_AUU_conf_cart = sp.sympify(0)
    for i in range(DIM):
        for j in range(DIM):
            ADD_times_AUU_conf_cart += ADD_conf_cart[i][j]*ADD_conf_cart[i][j]
    return ADD_times_AUU_conf_cart

<a id='psi_singular'></a>

# Step 4: Computing $\psi_{\rm singular}$ \[Back to [top](#toc)\]
$$\label{psi_singular}$$

Computing the singular piece of the conformal factor, which is given by:
\begin{equation}
\psi_{\rm singular} = 1 + \sum_{n=1}^{2} \frac{m_{n}}{2|\vec{x}_{n} |} \,.
\end{equation}

Note: In the NRPyElliptic code, the singular piece of the conformal factor is referred to as `psi_background`.

In [11]:
def psi_background_cartesian (bare_mass_0, xU0, bare_mass_1, xU1):
    """
    Inputs: 
        - bare mass of each puncture
        - xnU = relative position of n-th puncture

    Output: background conformal factor in Cartesian coordinates
    """
    # Set grid dimension
    DIM = par.parval_from_str("DIM")
    
    def psi_background_cartesian_single_puncture (bare_mass, xU):
        # Compute distance of puncture from the origin
        r = sp.sympify(0)
        for i in range(DIM):
            r += sp.Pow(xU[i], 2)
        r = sp.sqrt(r)
        
        return (bare_mass/(2*r))
    
    psi_puncture_0 = psi_background_cartesian_single_puncture (bare_mass_0, xU0)
    psi_puncture_1 = psi_background_cartesian_single_puncture (bare_mass_1, xU1)
    
    return (1 + psi_puncture_0 + psi_puncture_1)

<a id='rhss'></a>

# Step 5: Computing the RHSs \[Back to [top](#toc)\]
$$\label{rhss}$$

Recall that the hyperbolized Hamiltonian constraint equation can be written as a first-order system:
\begin{equation}
  \boxed{
    \begin{aligned}
      \partial_{t}u &= v - \eta u\\
      \partial_{t}v &= c^{2}\biggl[\ttilde\nabla^{2}u + \frac{1}{8}\ttilde{A}_{ij}\ttilde{A}^{ij}\bigl(\psi_{\rm singular}+u\bigr)^{-7}\biggr]
    \end{aligned}
  }\ .
\end{equation}
Note that, for conformally flat spaces, $\ttilde{\gamma}_{ij} = \hat{g}_{ij}$, which is simply the flat metric in the chosen coordinate system.

Next, we implement these equations.

In [12]:
# Declare damping parameter
eta_damping = par.Cparameters("REAL",thismodule,"eta_damping", 1.0)
# Declare spatially-dependent wavespeed as grid function
wavespeed = gri.register_gridfunctions("AUXEVOL", ["wavespeed"])
# Declare psi_background and ADD_times_AUU
psi_background, ADD_times_AUU = gri.register_gridfunctions("AUXEVOL",["psi_background","ADD_times_AUU"])

The Laplacian operator is computed as follows:
$$
\hat\nabla^{2}u  =
{\underbrace {\textstyle \hat{g}^{ij} \partial_{i} \partial_{j} u}_{\text{Part 1}}} 
{\underbrace {\textstyle -\hat{\Gamma}^i \partial_i u}_{\text{Part 2}}},
$$
where $\hat{\Gamma}^i = \hat{g}^{ij}\hat{\Gamma}^k_{ij}$ are the contracted Christoffel symbols.

In [13]:
# Step 1: Compute the contracted Christoffel symbols:
contractedGammahatU = ixp.zerorank1()
for k in range(DIM):
    for i in range(DIM):
        for j in range(DIM):
            contractedGammahatU[k] += rfm.ghatUU[i][j] * rfm.GammahatUDD[k][i][j]

# Step 2: Register gridfunctions that are needed as input
#         to the scalar wave RHS expressions.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 3a: Declare the rank-1 indexed expression \partial_{i} u,
#          Derivative variables like these must have an underscore
#          in them, so the finite difference module can parse the
#          variable name properly.
uu_dD = ixp.declarerank1("uu_dD")

# Step 3b: Declare the rank-2 indexed expression \partial_{ij} u,
#          which is symmetric about interchange of indices i and j
#          Derivative variables like these must have an underscore
#          in them, so the finite difference module can parse the
#          variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym01")

# Step 4: Define right-hand sides for the evolution.
uu_rhs = vv - eta_damping*uu
# Step 4b: The right-hand side of the \partial_t v equation
#          is given by:
#          \hat{g}^{ij} \partial_i \partial_j u - \hat{\Gamma}^i \partial_i u.
#          ^^^^^^^^^^^^ PART 1 ^^^^^^^^^^^^^^^^ ^^^^^^^^^^ PART 2 ^^^^^^^^^^^
vv_rhs = sp.sympify(0)
for i in range(DIM):
    # PART 2:
    vv_rhs -= contractedGammahatU[i]*uu_dD[i]
    for j in range(DIM):
        # PART 1:
        vv_rhs += rfm.ghatUU[i][j]*uu_dDD[i][j]

# Add source term
vv_rhs += sp.Rational(1,8)*ADD_times_AUU*sp.Pow(psi_background + uu, -7)

# Include wavespeed 
vv_rhs *= wavespeed*wavespeed

<a id='code_validation'></a>

# Step 6: Code validation against `NRPyElliptic.NRPyElliptic_SourceTerm` and `NRPyElliptic.NRPyElliptic_RHSs`  NRPy+ modules \[Back to [top](#toc)\]
$$\label{code_validation}$$

We can validate the expressions implemented in this notebook against the trusted NRPy+ modules of the NRPyElliptic code. We will use the `check_zero()` function from NRPy+'s unit test suite. This functions checks if a symbolic expression equals $0$ by injecting several random numbers and computing the result using arbitrary-precision arithmetic.

In [14]:
from UnitTesting.assert_equal import check_zero # NRPy+: Symbolic expression verification

<a id='code_validation_sourceterm'></a>

## Step 6.a: Code validation against `NRPyElliptic.NRPyElliptic_SourceTerm` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation_sourceterm}$$

In [15]:
# Import source term generation module
import NRPyElliptic_codegen.NRPyElliptic_SourceTerm as puncsource

In [16]:
# Compute singular piece of conformal metric (psi_background) and contraction of extrinsic curvature
# Fisrt, using the imported module
puncsource.PunctureInitialDataCurvilinear_psi_background_and_ADD_AUU_term()

In [17]:
# Now with the expressions coded up in this notebook

# Define vector field V_i
VU_cart = VU_cart_two_punctures(xU[0], PU[0], SU[0], xU[1], PU[1], SU[1])

# Step Define extrinsic curvature
ADD_conf_cart = ADD_conf_cartesian(VU_cart)

# Step Compute the scalar ADD*AUU in Cartesian coordinates
ADD_times_AUU_conf_cart = ADD_times_AUU_conf_cartesian(ADD_conf_cart)

# Define function to replace xxCart by the Cartesian coordinates expressed in curvilinear coordinates
def replace_cart_coord_by_xx (src, DIM=3):
    for k in range(DIM):
        src = src.subs(rfm.Cart[k],rfm.xx_to_Cart[k])
    return src

# Step(?): Compute psi_background
psi_background_cart = psi_background_cartesian (bare_mass_0, xU[0], bare_mass_1, xU[1])
_psi_background = replace_cart_coord_by_xx(psi_background_cart)

# Step(?): Compute ADD_times_AUU
_ADD_times_AUU = replace_cart_coord_by_xx(ADD_times_AUU_conf_cart)

In [18]:
# Check if expressions are in agreement
if check_zero(_psi_background - puncsource.psi_background):
    print("psi_background validation test passed!")
else:
    print("psi_background validation test failed!")
    sys.exit(1)
if check_zero(_ADD_times_AUU - puncsource.ADD_times_AUU):
    print("ADD_times_AUU validation test passed!")
else:
    print("ADD_times_AUU validation test failed!")
    sys.exit(1)

psi_background validation test passed!
ADD_times_AUU validation test passed!


<a id='code_validation_rhss'></a>

## Step 6.b: Code validation against `NRPyElliptic.NRPyElliptic_RHSs` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation_rhss}$$

In [19]:
# Import right-hand sides module.
import NRPyElliptic_codegen.NRPyElliptic_RHSs as puncrhs
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []
# Generate RHSs
puncrhs.PunctureInitialDataCurvilinear_RHSs()

In [20]:
# Check if expressions are in agreement
if check_zero(uu_rhs - puncrhs.uu_rhs):
    print("uu_rhs validation test passed!")
else:
    print("uu_rhs validation test failed!")
    sys.exit(1)
if check_zero(vv_rhs - puncrhs.vv_rhs):
    print("vv_rhs validation test passed!")
else:
    print("vv_rhs validation test failed!")
    sys.exit(1)

uu_rhs validation test passed!
vv_rhs validation test passed!


<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-NRPyElliptic_BasicEquations.pdf](Tutorial-NRPyElliptic_BasicEquations.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [21]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-NRPyElliptic_BasicEquations")

Created Tutorial-NRPyElliptic_BasicEquations.tex, and compiled LaTeX file
    to PDF file Tutorial-NRPyElliptic_BasicEquations.pdf
